In [16]:
import pickle
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

from tensorflow import keras
from pathlib import Path

%matplotlib inline

In [5]:
DATA = Path('./data')

In [ ]:
!wandb login

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode = 'fine')

In [9]:
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [3]:
# lets pickle the data so we can store it in a bucket
data = [x_train, y_train, x_test, y_test]
with open(DATA/'training_data.dat', "wb") as f:
    pickle.dump(data, f)

In [11]:
with open(DATA/'training_data.dat', "rb") as f:
    [x_train, y_train, x_test, y_test] = pickle.load(f)

In [12]:
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
# from: https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/Simple_Keras_Integration.ipynb#scrollTo=_Mqy_ZBNM8UA
run = wandb.init(project='mlops-midterm',
                 config={
                     "learning_rate": 0.005,
                     "epochs": 10,
                     "batch_size": 64,
                     "loss_function": "sparse_categorical_crossentropy",
                     "metrics":["sparse_categorical_accuracy"],
                     "optimizer": "adam",
                     "base": "EfficientNetB0",
                     "pool": "avg",
                     "head": "Dense(100)",
                     "dataset": "CIFAR-100"
                 })
config = wandb.config

In [24]:
base = keras.applications.EfficientNetB0(
    include_top = False,
    weights = 'imagenet',
    input_shape = [32, 32, 3],
    pooling = config.pool
)
base.trainable = False

In [28]:
#base.summary()

Build and compile model

In [26]:
model = keras.Sequential([
    base,
    keras.layers.Dense(100, activation = 'sigmoid')
])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               128100    
Total params: 4,177,671
Trainable params: 128,100
Non-trainable params: 4,049,571
_________________________________________________________________


In [29]:
optimizer = tf.keras.optimizers.Adam(config.learning_rate)
model.compile(
    optimizer = optimizer,
    loss = config.loss,
    metrics = config.metrics
)

In [ ]:
model.fit(
    x = x_train,
    y = y_train,
    validation_data = (x_test, y_test),
    epochs = config.epochs,
    callbacks = [WandbCallback]
)

In [18]:
with open('cfg_00001.json', 'r') as f:
    config = json.load(f)

In [20]:
config

{'data_set': 'training_data.dat',
 'learning_rate': 0.005,
 'epochs': 10,
 'batch_size': 64,
 'loss_function': 'sparse_categorical_crossentropy',
 'metrics': ['sparse_categorical_accuracy'],
 'optimizer': 'adam',
 'base': 'EfficientNetB0',
 'pool': 'avg',
 'head': 'Dense(100)',
 'dataset': 'CIFAR-100'}

In [21]:
config.metrics

AttributeError: 'dict' object has no attribute 'metrics'